In [42]:
pip install transformers torch 

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in e:\university\tlcn\chatbox\venv\lib\site-packages (4.43.4)



You should consider upgrading via the 'e:\university\TLCN\ChatBox\venv\Scripts\python.exe -m pip install --upgrade pip' command.


### 1. Import thư viện

In [26]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tranl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 2. Load data

In [53]:

df = pd.read_csv('../csv/processed_medical_tf-idf.csv')
df


,Title,Detailed Content,Reference Link,Answer_of_Title,Answer_Start,Answer_End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...,các ngoại_lệ bao_gồm virus papilloma_người có_...,144,501
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...,ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệ...,62,519
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...,đường hô_hấp có các hệ_thống lọc đường thở trên,0,48
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,0,230
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,0,318
...,...,...,...,...,...,...
23546,hồi_sức tĩnh_mạch về tài_liệu tham_khảo về dịch,1 . transfusion of plasma platelets and red bl...,https://www.msdmanuals.com/vi-vn/professional/...,doi 101001 jama 201512 2 myburgh ja finfer s b...,602,778
23547,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...,áp_suất tĩnh_mạch trung_tâm cvp là áp_suất tru...,164,307
23548,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,bệnh_nhân bị sốc mất máu do chấn_thương có_thể...,https://www.msdmanuals.com/vi-vn/professional/...,bệnh_nhân bị sốc mất máu do chấn_thương có_thể...,0,73
23549,hồi_sức tĩnh_mạch về áp_lực tĩnh_mạch trung_tâm,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...,áp_suất tĩnh_mạch trung_tâm cvp là áp_suất tru...,164,307


In [54]:
df.loc[1, 'Detailed Content']

'nhiều màng_nhầy ngập trong chất tiết có đặc_tính kháng khuẩn . ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệt và nước_mắt có chứa lysozyme chất này chia_cắt liên_kết axit muramic trong thành tế_bào vi_khuẩn đặc_biệt là ở các sinh_vật gram dương vi_khuẩn gram âm được bảo_vệ bởi lipopolysaccharid ở màng ngoài của chúng . các dịch tiết tại_chỗ cũng chứa globulin miễn_dịch chủ_yếu là igg và iga tiết ra ngăn_ngừa các vi_sinh_vật gắn vào tế_bào chủ và các protein liên_kết sắt rất cần_thiết cho nhiều vi_sinh_vật .'

##### Xử lý các giá trị null và duplicate

In [55]:
df.isnull().sum()

Title               0
Detailed Content    0
Reference Link      0
Answer_of_Title     0
Answer_Start        0
Answer_End          0
dtype: int64

In [56]:
# Giả sử df là DataFrame của bạn
# Kiểm tra số lượng giá trị trùng lặp trong cột 'Title'
duplicates_count_title = df['Title'].duplicated().sum()
duplicates_count_content = df['Detailed Content'].duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'Title': {duplicates_count_title}")
print(f"Số lượng giá trị trùng lặp trong cột 'Detailed Content': {duplicates_count_content}")



Số lượng giá trị trùng lặp trong cột 'Title': 1436
Số lượng giá trị trùng lặp trong cột 'Detailed Content': 10200


In [57]:
# Tìm các title bị trùng lặp
duplicate_titles = df[df.duplicated(['Title'], keep=False)]

# Hàm để lấy từ ghép đầu tiên từ content
def get_first_compound_word(text):
    # Giả định rằng các từ ghép được phân tách bằng dấu '_'
    words = text.split()
    for word in words:
        if '_' in word:
            return word
    return ''

# Duyệt qua các title trùng lặp và cập nhật lại title
for index, row in duplicate_titles.iterrows():
    first_compound_word = get_first_compound_word(row['Detailed Content'])
    if first_compound_word:
        new_title = f"{row['Title']} theo {first_compound_word}"
        df.at[index, 'Title'] = new_title

# Kiểm tra lại số lượng giá trị trùng lặp sau khi cập nhật
duplicates_count_title_after = df['Title'].duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'Title' sau khi cập nhật: {duplicates_count_title_after}")

Số lượng giá trị trùng lặp trong cột 'Title' sau khi cập nhật: 1117


In [58]:
df = df.drop_duplicates(subset=['Detailed Content'])
df =  df.dropna(axis=1, how='all')

In [59]:
df = df.drop_duplicates()

# Tìm các giá trị trùng lặp

In [60]:
# Tìm các hàng trùng lặp
duplicates = df[df.duplicated(subset='Title', keep='first')]

# Hiển thị các hàng trùng lặp
print("Các hàng trùng lặp:")
duplicates

Các hàng trùng lặp:


,Title,Detailed Content,Reference Link,Answer_of_Title,Answer_Start,Answer_End
174,chống chỉ_định và thận_trọng khi tiêm về chống...,chống chỉ_định chính của vắc xin viêm gan b là...,https://www.msdmanuals.com/vi-vn/professional/...,chống chỉ_định chính của vắc xin viêm gan b là...,0,166
175,chống chỉ_định và thận_trọng khi tiêm về thận_...,các thận_trọng chính vắc xin hepb là bệnh nhẹ ...,https://www.msdmanuals.com/vi-vn/professional/...,các thận_trọng chính vắc xin hepb là bệnh nhẹ ...,0,130
221,chống chỉ_định và thận_trọng khi tiêm về chống...,chống chỉ_định đối_với vắc xin ngừa rotavirus ...,https://www.msdmanuals.com/vi-vn/professional/...,chống chỉ_định đối_với vắc xin ngừa rotavirus ...,0,269
326,chẩn_đoán nhiễm ký_sinh_trùng về huyết_thanh h...,huyết_thanh hoặc csf kháng_nguyên dùng để đánh...,https://www.msdmanuals.com/vi-vn/professional/...,huyết_thanh hoặc csf kháng_nguyên dùng để đánh...,0,106
410,lựa_chọn và sử_dụng kháng_sinh về đối_với trẻ ...,đối_với trẻ lớn viên 125 – 250 mg viên nén 12 ...,https://www.msdmanuals.com/vi-vn/professional/...,đối_với trẻ lớn viên 125 – 250 mg viên nén 12 ...,0,57
...,...,...,...,...,...,...
23180,mô_tả các bước_tiến_hành thủ_thuật về luồn dây...,cẩn_thận xoay ống_tiêm của dụng_cụ đặt ống thô...,https://www.msdmanuals.com/vi-vn/professional/...,sau đó sử_dụng gạc vuông 4 × 4 trong 10 phút é...,1226,1865
23183,mô_tả các bước_tiến_hành thủ_thuật về đặt ống ...,đẩy ống thông_qua dây_dẫn hướng đến bề_mặt da ...,https://www.msdmanuals.com/vi-vn/professional/...,nếu nhịp tim ngoại tâm thu xảy ra từ từ rút ốn...,709,1160
23185,mô_tả các bước_tiến_hành thủ_thuật về sử_dụng ...,sử_dụng dẫn hướng siêu_âm trục ngắn lấy hình_ả...,https://www.msdmanuals.com/vi-vn/professional/...,khi sử_dụng dẫn hướng siêu_âm trục ngắn chọc k...,563,773
23325,căn_nguyên của ngừng thở về tắc_nghẽn đường th...,tắc_nghẽn đường thở trên có_thể xảy ra ở trẻ s...,https://www.msdmanuals.com/vi-vn/professional/...,tắc_nghẽn đường thở trên có_thể xảy ra ở trẻ s...,0,159


### Xóa tài liệu tham khảo

In [61]:
# Lọc các tiêu đề có chứa 'Tài liệu tham khảo'
filtered_df = df[df['Title'].str.contains('tài_liệu tham_khảo', case=False, na=False)]

# In ra kết quả
filtered_df

,Title,Detailed Content,Reference Link,Answer_of_Title,Answer_Start,Answer_End
90,đánh_giá sốt không rõ nguyên_nhân về tài_liệu ...,1 . lack of clinical utility of labeled white ...,https://www.msdmanuals.com/vi-vn/professional/...,doi 101093 ofid ofac015 1 fisher re drews_al p...,155,362
125,tiêm_chủng vắc xin về tài_liệu tham_khảo về vắ...,1 . a needling problem shoulder injury related...,https://www.msdmanuals.com/vi-vn/professional/...,doi 103122 jabfm 201206110334,118,151
154,an_toàn vắc xin về tài_liệu tham_khảo về an_to...,1 . vaccines and autism a tale of shifting hyp...,https://www.msdmanuals.com/vi-vn/professional/...,pmid 28671426,313,327
178,chế_phẩm vắc xin hpv về tài_liệu tham_khảo về ...,1 . human papillomavirus hpv vaccines . 2019 ....,https://www.msdmanuals.com/vi-vn/professional/...,1 human papillomavirus hpv vaccines,0,41
252,chẩn_đoán tiêu_chảy do về tài_liệu tham_khảo c...,1 . acg clinical guidelines prevention diagnos...,https://www.msdmanuals.com/vi-vn/professional/...,1 acg clinical guidelines prevention diagnosis...,0,520
...,...,...,...,...,...,...
23485,điều_trị nhiễm_trùng huyết và sốc nhiễm_trùng ...,1 . identifying patients with sepsis on the ho...,https://www.msdmanuals.com/vi-vn/professional/...,1 identifying patients with sepsis on the hosp...,0,2028
23499,sinh_lý bệnh của sốc về tài_liệu tham_khảo sin...,1 . endothelial glycocalyx dysfunction in dise...,https://www.msdmanuals.com/vi-vn/professional/...,1 endothelial glycocalyx dysfunction in diseas...,0,132
23517,chẩn_đoán sốc về tài_liệu tham_khảo chẩn_đoán,1 . image based resuscitation of the hypotensi...,https://www.msdmanuals.com/vi-vn/professional/...,1 image based resuscitation of the hypotensive...,0,848
23526,điều_trị sốc về tài_liệu tham_khảo về điều_trị,1 . transfusion of plasma platelets and red bl...,https://www.msdmanuals.com/vi-vn/professional/...,1 transfusion of plasma platelets and red bloo...,0,197


In [62]:
# Lọc các tiêu đề chứa 'Tài liệu tham khảo'
mask = df['Title'].str.contains('tài_liệu tham_khảo', case=False, na=False)

# Xóa các hàng chứa tiêu đề với 'Tài liệu tham khảo'
df = df[~mask]
df = df.reset_index(drop=True)

# In ra kết quả
df

,Title,Detailed Content,Reference Link,Answer_of_Title,Answer_Start,Answer_End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...,các ngoại_lệ bao_gồm virus papilloma_người có_...,144,501
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...,ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệ...,62,519
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...,đường hô_hấp có các hệ_thống lọc đường thở trên,0,48
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,0,230
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,0,318
...,...,...,...,...,...,...
11887,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,https://www.msdmanuals.com/vi-vn/professional/...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,0,126
11888,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,máu thường được cung_cấp dưới dạng khối hồng_c...,https://www.msdmanuals.com/vi-vn/professional/...,bệnh_nhân_truyền 6 đơn_vị đòi_hỏi cần thay_thế...,280,465
11889,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,chất thay_thế máu là dịch mang oxy có_thể được...,https://www.msdmanuals.com/vi-vn/professional/...,chất thay_thế máu là dịch mang oxy có_thể được...,0,92
11890,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...,áp_suất tĩnh_mạch trung_tâm cvp là áp_suất tru...,164,307


#### 3. Tìm câu trả lời 

In [63]:
def split_into_sentences(text, max_sentence_length=2048):
    # Kiểm tra xem text có phải là chuỗi không và xử lý NaN
    if isinstance(text, str):
        # Tách văn bản theo dấu câu như ".", "!", "?" và giữ lại dấu câu
        sentences = re.split(r'(?<=[.!?]) +', text)
        
        # Nếu một câu dài hơn max_sentence_length, chia nhỏ nó thành các câu con
        split_sentences = []
        for sentence in sentences:
            while len(sentence) > max_sentence_length:
                # Tìm điểm dừng hợp lý trong câu (ví dụ, sau một dấu cách)
                split_point = sentence.rfind(' ', 0, max_sentence_length)
                if split_point == -1:  # Nếu không tìm thấy dấu cách
                    split_point = max_sentence_length
                split_sentences.append(sentence[:split_point])
                sentence = sentence[split_point:].strip()  # Lấy phần còn lại của câu
            split_sentences.append(sentence)
        
        return split_sentences
    else:
        # Trả về danh sách rỗng nếu text không phải là chuỗi hoặc là NaN
        return []


# Hàm tính toán TF-IDF, chọn đoạn có liên quan cao nhất, và tìm vị trí của câu trong ngữ cảnh
def extract_highest_tfidf_sentence_with_position(title, context):
    # Chia context thành các đoạn văn
    sentences = split_into_sentences(context)

    # Loại bỏ các câu trùng lặp bằng cách chuyển thành set, sau đó chuyển về danh sách
    unique_sentences = list(set(sentences))

    # Tạo danh sách kết hợp Title và các đoạn trong context để tính TF-IDF
    documents = [title] + unique_sentences  # Tiêu đề được đặt trước các đoạn văn trong context

    # Khởi tạo TfidfVectorizer để tính toán TF-IDF
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Tính điểm TF-IDF của từng đoạn văn dựa trên sự liên quan với tiêu đề
    title_vector = tfidf_matrix[0].toarray()  # Vector của title
    context_vectors = tfidf_matrix[1:].toarray()  # Vectors của các đoạn trong context

    # Tính độ tương đồng giữa title và từng đoạn context bằng cách tính tích vô hướng
    similarity_scores = np.dot(context_vectors, title_vector.T).flatten()

    # Tìm chỉ số của câu có điểm TF-IDF cao nhất
    highest_score_index = np.argmax(similarity_scores)

    # Lấy câu có điểm TF-IDF cao nhất
    highest_tfidf_sentence = unique_sentences[highest_score_index]

    # Tìm vị trí bắt đầu (answer_start) và kết thúc (answer_end) của câu trong context ban đầu
    answer_start = context.find(highest_tfidf_sentence)
    if answer_start != -1:
        answer_end = answer_start + len(highest_tfidf_sentence)
    else:
        # Nếu không tìm thấy câu trong context ban đầu (đề phòng các trường hợp không khớp hoàn toàn)
        answer_start = -1
        answer_end = -1

    return highest_tfidf_sentence, answer_start, answer_end



In [64]:
# Áp dụng hàm lên từng hàng của DataFrame và trả về 3 giá trị
df[['Answer_of_Title', 'Answer_Start', 'Answer_End']] = df.apply(
    lambda row: pd.Series(extract_highest_tfidf_sentence_with_position(row['Title'], row['Detailed Content'])),
    axis=1
)

# Hiển thị kết quả
df[['Title', 'Detailed Content', 'Answer_of_Title', 'Answer_Start', 'Answer_End']].head()


,Title,Detailed Content,Answer_of_Title,Answer_Start,Answer_End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,140,479
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệ...,63,321
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,đường hô_hấp có các hệ_thống lọc đường thở trên .,0,49
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,0,139
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,0,173


In [65]:
df[['Title', 'Detailed Content', 'Answer_of_Title', 'Answer_Start', 'Answer_End']].head(10)

,Title,Detailed Content,Answer_of_Title,Answer_Start,Answer_End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,140,479
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệ...,63,321
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,đường hô_hấp có các hệ_thống lọc đường thở trên .,0,49
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,0,139
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,0,173
5,các yếu_tố độc lực về vỏ,một_số sinh_vật ví_dụ một_số chủng phế cầu não...,tuy_nhiên các kháng_thể opsonic đặc_hiệu vỏ có...,188,298
6,các yếu_tố độc lực về enzyme,các protein vi_khuẩn có hoạt_tính enzyme ví_dụ...,các protein vi_khuẩn có hoạt_tính enzyme ví_dụ...,0,140
7,các yếu_tố độc lực về độc_chất,các sinh_vật có_thể giải_phóng chất_độc gọi là...,hầu_hết các chất_độc gắn với các thụ thể đặc_h...,198,265
8,các yếu_tố độc lực về các yếu_tố khác,một_số vi_sinh_vật có tính độc_hại hơn bởi_vì ...,tính đề_kháng khả_năng ly giải của bổ thể góp_...,634,698
9,các yếu_tố độc lực về nội độc_tố,nội độc_tố là một lipopolysaccharide được tạo ...,nội độc_tố khởi_động các đáp_ứng dịch_thể liên...,125,291


# Lưu data

In [66]:
# Lưu DataFrame vào file CSV
df.to_csv('../csv/processed_medical_cleaned.csv', index=False, encoding='utf-8')

# Show detail dataset

In [67]:
df

,Title,Detailed Content,Reference Link,Answer_of_Title,Answer_Start,Answer_End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...,các ngoại_lệ bao_gồm virus papilloma_người có_...,140,479
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...,ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệ...,63,321
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...,đường hô_hấp có các hệ_thống lọc đường thở trên .,0,49
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,0,139
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,0,173
...,...,...,...,...,...,...
11887,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,https://www.msdmanuals.com/vi-vn/professional/...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,0,122
11888,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,máu thường được cung_cấp dưới dạng khối hồng_c...,https://www.msdmanuals.com/vi-vn/professional/...,bệnh_nhân_truyền 6 đơn_vị đòi_hỏi cần thay_thế...,274,457
11889,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,chất thay_thế máu là dịch mang oxy có_thể được...,https://www.msdmanuals.com/vi-vn/professional/...,chất thay_thế máu là dịch mang oxy có_thể được...,0,93
11890,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...,áp_suất tĩnh_mạch trung_tâm cvp là áp_suất tru...,164,305


In [76]:
# Lấy nội dung của cột "Content" ở dòng đầu tiên
first_row_title = df.loc[11888, 'Title']
first_row_content = df.loc[11888, 'Detailed Content']

# In nội dung của cột "Content" ở dòng đầu tiên
print("Content of the first row:")
print(first_row_title)    
print(first_row_content)    

Content of the first row:
dịch để hồi_sức bằng dịch qua đường tĩnh_mạch về máu
máu thường được cung_cấp dưới dạng khối hồng_cầu cần phải được thử máu chéo nhưng trong trường_hợp khẩn_cấp 1 đến 2 đơn_vị máu nhóm o_rh âm_tính là một lựa_chọn thay_thế có_thể chấp_nhận được . truyền 1 đến 2 đơn_vị ví_dụ trong chấn_thương lớn máu được làm nóng đến 37 ° c. bệnh_nhân_truyền 6 đơn_vị đòi_hỏi cần thay_thế các yếu_tố đông máu bằng truyền tĩnh_mạch huyết_tương tươi đông_lạnh hoặc cryoprecipitate và truyền tiểu_cầu xem thêm các sản_phẩm máu .


In [45]:
def print_max_length_rows(df, columns):
    max_length_info = {}
    
    for column in columns:
        # Tính toán chiều dài của từng văn bản trong cột
        df[f'{column}_length'] = df[column].apply(lambda x: len(str(x)))  # Thêm cột độ dài
        
        # Tìm hàng có độ dài tối đa
        max_length_row = df.loc[df[f'{column}_length'].idxmax()]
        
        # Lưu thông tin vào từ điển
        max_length_info[column] = {
            'index': max_length_row.name,
            'length': max_length_row[f'{column}_length'],
            'value': max_length_row[column]
        }
    
    # In kết quả cuối cùng
    for column, info in max_length_info.items():
        print(f"Max length in '{column}':")
        print(f"Index: {info['index']}")
        print(f"Length: {info['length']}")
        print(f"Value: {info['value']}")
        print('-' * 40)

# Xác định các cột cần kiểm tra
columns_to_check = ['Title', 'Answer_of_Title']

# In hàng có độ dài tối đa trong từng cột
print_max_length_rows(df, columns_to_check)

Max length in 'Title':
Index: 90
Length: 248
Value: liều_lượng và cách sử_dụng vắc xin phòng covid 19 về đối_với các loại vắc xin ngừa covid 19 có sẵn để sử_dụng ở hoa_kỳ tất_cả thông_tin về liều_lượng và cách sử_dụng cho mọi nhóm tuổi và những người có tình_trạng đặc_biệt đều có sẵn từ cdc tại phần
----------------------------------------
Max length in 'Answer_of_Title':
Index: 5767
Length: 2047
Value: gia_đình bệnh_nhân hoặc các thành_viên cộng_đồng tiếp_xúc gần_gũi có xuất_hiện các triệu_chứng tương_tự triệu_chứng nặng hơn về buổi đêm khám lâm_sàng soi kính hiển_vi ở vùng da trầy_xước để tìm hang_ghẻ mày_đay các tổn_thương hồng ban dạng vòng nổi gồ lên với trung_tâm nhạt màu hơn có_thể là cấp_tính 6 tuần hoặc mạn tính 6 tuần chỉ khám lâm_sàng khô_da da khô phổ_biến nhất trong mùa đông da ngứa khô vảy da chủ_yếu ở các chi dưới trầm_trọng hơn do khô nóng chỉ khám lâm_sàng rối_loạn hệ_thống phản_ứng dị_ứng bên trong ăn nhiều thực_phẩm ngứa toàn_thân phát_ban với các dát hoặc nốt sẩn ho